In [1]:
import numpy as np
import sys
import os
import neuro
import risp
import random

from encode import Chromagram
from encode import Encoder
from data_loader import data_loader
import risp
import neuro
import eons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [2]:
risp_config = {
  "leak_mode": "all",
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5,
  "discrete": False
} 
proc = risp.Processor(risp_config) # RISP processor

net = neuro.Network() # Neuro network
net.set_properties(proc.get_network_properties()) # Set network properties

n_inputs = 12
n_hidden = 360
n_outputs = 12
n_neurons = n_inputs + n_hidden + n_outputs
n_synapses = 1000


moa = neuro.MOA()
moa.seed(42)


In [3]:
def create_neuron(neuron_id, net, moa):
    neuron = net.add_node(neuron_id)
    net.randomize_node_properties(moa, neuron)
    return neuron

for i in range(n_inputs):
    neuron = create_neuron(i, net, moa)
    neuron.set("Threshold",0.75)
    net.add_input(i)
    
for i in range(n_outputs):
    neuron = create_neuron(i+n_inputs, net, moa)
    neuron.set("Threshold",0.75)
    net.add_output(i)
    
for i in range(n_hidden):
    neuron = create_neuron(i+n_inputs+n_outputs, net, moa)

In [4]:
# First, split the data into training+validation and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(data_loader().chroma_files, data_loader().numerical_label_roots, test_size=0.2, random_state=42)

# Now, split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# One-hot encode the labels
OneHot = OneHotEncoder(sparse_output=False)
y_train_encoded = OneHot.fit_transform(np.array(y_train).reshape(-1, 1))
y_val_encoded = OneHot.transform(np.array(y_val).reshape(-1, 1))
y_test_encoded = OneHot.transform(np.array(y_test).reshape(-1, 1))

In [5]:
len(X_train)

2904

In [6]:
X_train[0]

'/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/sus4/MODX8_sus4_C#3_49.npy'

In [7]:
encoder = Encoder(X_train[2])

len(encoder.spikes)

5015

In [8]:
proc.load_network(net)

True

In [9]:
for i in range(n_outputs):
    proc.track_output_events(i)
    print(proc.track_output_events(i))

True
True
True
True
True
True
True
True
True
True
True
True


In [10]:
proc.apply_spikes(encoder.spikes)

In [11]:
proc.run(100)

In [12]:
vectors = proc.output_vectors()
print(vectors)

for item in vectors:
    print(len(item))

[[0.0, 6.0, 12.0, 18.0, 23.0, 30.0, 38.0, 45.0, 46.0, 55.0, 65.0, 69.0, 79.0, 89.0, 92.0], [0.0, 4.0, 9.0, 13.0, 18.0, 22.0, 23.0, 29.0, 35.0, 41.0, 46.0, 54.0, 63.0, 69.0, 77.0, 85.0, 92.0, 99.0], [0.0, 2.0, 5.0, 8.0, 10.0, 13.0, 16.0, 18.0, 21.0, 23.0, 27.0, 30.0, 34.0, 38.0, 42.0, 46.0, 54.0, 62.0, 69.0, 75.0, 81.0, 88.0, 92.0, 99.0], [0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 17.0, 19.0, 21.0, 23.0, 25.0, 27.0, 28.0, 30.0, 32.0, 34.0, 36.0, 38.0, 40.0, 42.0, 44.0, 46.0, 47.0, 49.0, 50.0, 52.0, 53.0, 55.0, 56.0, 58.0, 59.0, 61.0, 62.0, 64.0, 65.0, 67.0, 68.0, 69.0, 70.0, 72.0, 73.0, 75.0, 76.0, 77.0, 79.0, 80.0, 81.0, 83.0, 84.0, 86.0, 87.0, 88.0, 90.0, 91.0, 92.0, 94.0, 95.0, 96.0, 97.0, 99.0], [0.0, 4.0, 8.0, 12.0, 16.0, 20.0, 23.0, 28.0, 34.0, 40.0, 45.0, 46.0, 56.0, 66.0, 69.0, 78.0, 86.0, 92.0], [0.0, 3.0, 7.0, 11.0, 15.0, 19.0, 23.0, 28.0, 33.0, 38.0, 42.0, 46.0, 56.0, 66.0, 69.0, 77.0, 84.0, 91.0, 92.0, 99.0], [0.0, 3.0, 6.0, 10.0, 13.0, 16.0, 20.0, 23.0, 27.0, 32.0, 36.0, 4

In [22]:
vector = proc.output_vector(0)
#make this list to a numpy array

vector

[0.0,
 6.0,
 12.0,
 18.0,
 23.0,
 30.0,
 38.0,
 45.0,
 46.0,
 55.0,
 65.0,
 69.0,
 79.0,
 89.0,
 92.0]

In [14]:
predicted_index = proc.output_count_max(12)[0]


In [23]:
proc.output_counts()

[15, 18, 24, 62, 18, 20, 25, 18, 16, 56, 22, 11]

In [15]:
output_count = np.array(proc.output_counts())
predicted_index = np.argmax(output_count)
print(predicted_index)

3


In [18]:
one_hot_prediction = np.zeros(n_outputs)
one_hot_prediction

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
one_hot_prediction[predicted_index]=1

In [20]:
one_hot_prediction

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])